<a href="https://colab.research.google.com/github/WHU-Peter/COMP6248-Deep-Learning/blob/master/4_4_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 4: Using GPU acceleration with PyTorch

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 22.2MB/s 


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [ ]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[ 1.7492,  1.0070, -1.8333,  ..., -1.0111, -0.3451,  0.0504],
        [ 0.2780,  0.2335, -0.0096,  ...,  0.2043, -0.2570, -0.8791],
        [ 0.4152,  2.4127,  1.2269,  ...,  0.1278, -0.0120,  3.4212],
        ...,
        [ 0.5435, -0.6110, -0.6374,  ..., -0.6285,  0.7729, -0.0612],
        [ 0.1227, -0.6610, -0.1518,  ..., -0.4352,  0.4924,  0.3540],
        [ 0.0068,  0.0486, -0.2152,  ..., -0.3117, -0.4037,  1.3873]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [ ]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[-0.0758, -0.5023,  0.8288,  ..., -0.5922, -0.1891,  0.0758],
        [-0.0668, -0.3299,  1.0997,  ...,  0.3886,  0.0391, -0.4952],
        [ 0.2116,  0.1812, -2.3418,  ...,  0.0689, -0.4592,  0.0818],
        ...,
        [-0.0211, -0.7105,  1.0150,  ..., -1.8255,  0.8644,  0.3163],
        [-0.2693, -0.6295, -0.0531,  ...,  1.4169,  1.9499, -0.0882],
        [-0.0068,  1.6970, -0.2313,  ..., -0.2989, -0.6553,  0.2052]],
       device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[ 1.1503,  0.0205,  0.1496,  ...,  0.0568, -1.9479,  0.0528],
        [-0.8327,  1.1295, -1.7157,  ..., -0.3567,  0.0697, -0.3672],
        [-0.1083,  0.2854,  0.0331,  ..., -1.7861, -0.3300,  0.6549],
        ...,
        [ 0.7326,  0.0589,  0.2680,  ..., -0.6204,  0.0377, -2.0415],
        [-0.3836,  1.9378,  0.3322,  ..., -0.9942,  0.8465, -2.9779],
        [ 0.2114,  1.1475,  0.5182,  ..., -0.1953,  0.1423, -1.3264]],
       device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

YOUR ANSWER HERE

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

YOUR ANSWER HERE